In [150]:
# 최종 코드 
import csv 
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 

# 1. 네이버 지도 키기, "강남 헤어샵 검색"
browser = webdriver.Chrome()
browser.get("https://m.map.naver.com/search2/search.naver?query=%EA%B0%95%EB%82%A8%20%ED%97%A4%EC%96%B4%EC%83%B5&sm=hty&style=v5")

# 2. 헤어샵 이름 검색 
shops = []
hair_shops = browser.find_elements(By.CSS_SELECTOR, '.item_tit')
for s in hair_shops[:3]: #가게 총 20개만 추가 
    shops.append(s.text)

# 10초 대기 
browser.implicitly_wait(3)

# csv 파일 생성
f = open('naver.csv', 'w', encoding="utf-8-sig", newline='')
w = csv.writer(f)
w.writerow(['헤어샵 이름', '작성자', '방문횟수', '리뷰'])


for j in range(len(shops)):
    total_reviews = []
    total_writers = []
    visit_nums = []
    
    # 3. 헤어샵 검색
    browser.get(f"https://m.map.naver.com/search2/search.naver?query={shops[j]}")
    
    browser.implicitly_wait(3) #대기 
    
    # 헤어샵 들어가기 
    shop_button = browser.find_element(By.CSS_SELECTOR, '#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview > div > strong')
    shop_button.click()
    
    # 4. 리뷰로 가는 란 클릭 
    review_button = browser.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(6) > span')
    review_button.click()
    
    browser.implicitly_wait(3) #대기 
    
    # 5. 더보기 버튼 4번 클릭 (총 50개의 리뷰)
    for i in range(2):
        more = browser.find_element(By.CLASS_NAME, 'fvwqf')
        more.click()
        browser.implicitly_wait(3) #대기 
    
    # Get scroll height
    SCROLL_PAUSE_TIME = 1
    last_height = browser.execute_script(
        "return document.body.scrollHeight")  #브라우져 높이를 확인 가능(자바스크립트)

    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);"
                              )  # 브라우져 끝까지 스크롤을 내리겠다.

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)  # 페이지 로딩 될 동안 웨잇

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 스크롤이 끝까지 내려가서 더이상 내릴 것이 없을 때
            try:
                browser.find_element_by_css_selector(
                    ".mye4qd").click()  # 검색어 더 찾아보기 클릭
            except:
                break
        last_height = new_height
    
    # 6. 리뷰 검색 및 text 추가
    reviews = browser.find_elements(By.CLASS_NAME, 'zPfVt')
    
    for i in reviews:
        total_reviews.append(i.text)
    
    # 7. 작성자 검색 및 text 추가 
    writers = browser.find_elements(By.CLASS_NAME, 'sBWyy')
    
    for i in writers:
        total_writers.append(i.text)
        
    # 8. 방문 횟수 검색 및 text 추가 
    nums = browser.find_elements(By.CLASS_NAME, 'P1zUJ')
    
    for i in nums[5::7]:
        visit_nums.append(i.text[0])
    
    # 9. 이미지는 어떻게 할건지 생각해봐야... 
    
    # 10. csv에 저장 
    for i in range(len(total_writers)):
        w.writerow([shops[j], total_writers[i], visit_nums[i], total_reviews[i]])
    
# 12. csv 파일 및 브라우저 닫기 
f.close()
browser.close() 

In [1]:
# ======================================== 이걸로 돌려야 =======================================================
# 함수 버전 
# 방문 횟수는 잘못 출력되는 경우가 많아서 일단 빼버림.. 
import csv 
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
from tqdm import tqdm 

def crawling(location):
    # 1. 네이버 지도 키기, "지역+헤어샵 검색"
    browser = webdriver.Chrome()
    wait = WebDriverWait(browser,10) # EC 기다리는 조건 
    browser.get(f"https://m.map.naver.com/search2/search.naver?query={location}헤어샵")
    # 2. 헤어샵 이름 검색 
    shops = []
    hair_shops = browser.find_elements(By.CSS_SELECTOR, '.item_tit')
    for s in hair_shops[:15]: #가게 총 15개만 검색
        shops.append(s.text)

    # 10초 대기 
    browser.implicitly_wait(10) # 10초 대기 

    # csv 파일 생성
    f = open(f'{location}.csv', 'w', encoding="utf-8-sig", newline='')
    w = csv.writer(f)
    w.writerow(['헤어샵 이름', '작성자', '리뷰', 'image_url'])


    for j in tqdm(range(len(shops))):
        total_reviews = []
        total_writers = []
        # visit_nums = []
        url = []

        # 3. 헤어샵 검색
        browser.get(f"https://m.map.naver.com/search2/search.naver?query={shops[j]}")

        browser.implicitly_wait(10) #대기 

        # 헤어샵 들어가기 
        shop_button = browser.find_element(By.CSS_SELECTOR, '#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview > div > strong')
        shop_button.click()
        browser.implicitly_wait(10) # 대기

        # 4. 리뷰로 가는 란 클릭 
        review_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(6) > span')))
        # review_button = browser.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(6) > span')
        if review_button.text != '리뷰': # 버튼이 리뷰가 아니면 다음 헤어샵으로 넘어감
            continue 
        review_button.click()

        # browser.implicitly_wait(10) #대기 

        # 5. 더보기 버튼 1번 클릭 (총 20개의 리뷰)
        for i in range(1):
            more = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'fvwqf')))
            # more = browser.find_element(By.CLASS_NAME, 'fvwqf')
            more.click()
            # browser.implicitly_wait(10) #대기 

        # 스크롤 내리기 (이거 안 내리면 더보기 한 매장들 데이터가 안 불러와짐)
        SCROLL_PAUSE_TIME = 1
        last_height = browser.execute_script(
            "return document.body.scrollHeight")  #브라우져 높이를 확인 가능(자바스크립트)

        while True:
            # Scroll down to bottom
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);"
                                  )  # 브라우져 끝까지 스크롤을 내리겠다.

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)  # 페이지 로딩 될 동안 wait

            # Calculate new scroll height and compare with last scroll height
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:  # 스크롤이 끝까지 내려가서 더이상 내릴 것이 없을 때
                try:
                    browser.find_element_by_css_selector(
                        ".mye4qd").click()  # 검색어 더 찾아보기 클릭
                except:
                    break
            last_height = new_height

        # 6. 리뷰 검색 및 text 추가
        reviews = browser.find_elements(By.CLASS_NAME, 'zPfVt')

        for i in reviews:
            total_reviews.append(i.text)

        # 7. 작성자 검색 및 text 추가 
        writers = browser.find_elements(By.CLASS_NAME, 'sBWyy')

        for i in writers:
            total_writers.append(i.text)
        '''
        # 8. 방문 횟수 검색 및 text 추가 
        nums = browser.find_elements(By.CLASS_NAME, 'P1zUJ')

        for i in nums[5::7]:
            visit_nums.append(i.text[0])
        '''
        # 9. 이미지 url 추가 
        image = browser.find_elements(By.CLASS_NAME, 'YeINN') 
        
        browser.implicitly_wait(10) # 10초 대기

        for i in range(len(image)): 
            try:
                s = image[i].find_element(By.CLASS_NAME, 'K0PDV').get_attribute('style').split('"')[1]
            except:
                s = "None" # 사진 없으면 None으로 추가
            finally:
                url.append(s)

        # 10. csv에 저장 
        count = min(len(total_writers), len(total_reviews), len(url)) # index out of range 나는 경우 있어서 처리
        for i in range(count):
            w.writerow([shops[j], total_writers[i], total_reviews[i], url[i]])

    # 12. csv 파일 및 브라우저 닫기 
    f.close()
    browser.close() 

In [5]:
# 총 1080개. (강남역=220, 홍대=140, 성수=210, 청담=130, 공덕=220, 사당=160)

In [15]:
crawling("강남역")

In [5]:
crawling("홍대")

100%|███████████████████████████████████████████| 15/15 [02:36<00:00, 10.40s/it]


In [4]:
crawling("성수")

100%|███████████████████████████████████████████| 15/15 [05:44<00:00, 22.99s/it]


In [2]:
crawling("청담")

100%|███████████████████████████████████████████| 15/15 [06:18<00:00, 25.26s/it]


In [3]:
crawling("공덕")

100%|███████████████████████████████████████████| 15/15 [15:16<00:00, 61.07s/it]


In [4]:
crawling("사당")

100%|███████████████████████████████████████████| 15/15 [12:30<00:00, 50.06s/it]
